# Analyzing environmental impacts of urban expansion in Phoeniz, Arizona (2017-2020)
Author: Kat Le


## About


## Highlights


## Data
2020 Tiger/Line shapefiles: https://www.census.gov/cgi-bin/geo/shapefiles/index.php?year=2020&layergroup=County+Subdivisions




In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import rioxarray as rioxr
from shapely.geometry import box
import contextily as ctx
from geogif import gif
import pystac_client
import stackstac

from pystac_client import Client # To access STAC catalogues

import planetary_computer # To sign items from MPC STAC Catalogue

from IPython.display import Image

### Preliminary exploration of Phoenix county spatial data


In [3]:
phoenix = gpd.read_file("../data/phoenix.shp")

In [ ]:
phoenix = phoenix.to_crs(epsg=3857)

In [ ]:
# Create the plot
fig, ax = plt.subplots(figsize=(10, 10))

# Turn off axis
#ax.set_axis_off()

# Plot the Phoenix data
phoenix.plot(ax=ax, facecolor='none', edgecolor='blue', 
             alpha=1, linewidth=5, zorder=2)

# Add a basemap from contextily 
# ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery)
ctx.add_basemap(ax, source=ctx.providers.Esri.WorldStreetMap)

# Add title
ax.set_title("Study Area: \n Phoenix Subdivision in Maricopa County, Arizona", size = 20)

# Show plot
plt.tight_layout()
plt.show()

### Access Biodiverstiy Intactness Index
We use the `Client` function from the `pystac_client` package to access the catalog:

In [ ]:
# Access MPC catalog
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

In [ ]:
bbox_of_interest = [-112.826843, 32.974108, -111.184387, 33.863574]

# Temporal range of interest
time_range = "2017-01-01/2020-01-01"

search = catalog.search(collections=["io-biodiversity"], 
                        bbox=bbox_of_interest,
                        datetime=time_range)

# Retrieve items
items = search.item_collection()
len(items)


In [ ]:
stack = (
    stackstac.stack(items, bounds_latlon=bbox_of_interest, assets=["data"])
    .assign_coords(
        time=pd.to_datetime([item.properties["start_datetime"] for item in items]).year
        .to_numpy()
    )
    .sortby("time")
)
stack.name = "Biodiversity Intactness"
stack

In [ ]:
bii = stack.squeeze().compute()

In [ ]:
bii = bii.rio.reproject(phoenix.crs)

In [ ]:
assert bii.rio.crs == phoenix.crs

In [ ]:
bii = bii.rio.clip(phoenix.geometry)

In [ ]:
# match crs to bii
# phoenix = phoenix.to_crs(bii.rio.crs)

In [ ]:
bii_2017 = bii.sel(time=2017)

above75_2017 = (bii_2017 >= 0.75)
above75_2017_count = above75_2017.sum().item()
total_cells = bii_2017.shape[0]*bii_2017.shape[1]

print(f"BII 2017 - Percent area above 75: {round((above75_2017_count / total_cells)*100, 2)}%")

In [ ]:
bii_2020 = bii.sel(time=2020)

above75_2020 = (bii_2020 >= 0.75)
above75_2020_count = above75_2020.sum().item()
total_cells = bii_2020.shape[0]*bii_2020.shape[1]

print(f"BII 2020 - Percent area above 75: {round((above75_2020_count / total_cells)*100, 2)}%")

In [ ]:
net_bii = above75_2020.astype(int) - above75_2017.astype(int)

In [ ]:
np.unique(net_bii)

In [ ]:
net_bii.plot()

In [ ]:
# # Create the plot
# fig, ax = plt.subplots(figsize=(10, 10))

# # Turn off axis
# ax.set_axis_off()

# # Add BII
# bii_2020.plot(ax=ax)

# # Add net BII
# #net_bii.plot(ax=ax)

# # Add title
# ax.set_title("Study Area: \n Phoenix County, Arizona", size = 20)

# # Show plot
# plt.tight_layout()
# plt.show()